In [153]:
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from CurrencyDataset import CurrencyDataset
from torch.utils.tensorboard.writer import SummaryWriter
from datetime import datetime
import math
import time 

In [2]:
df = pd.read_csv('../cleaned/usdrub_new.csv')

In [3]:
columns = ['close']
df = df[columns]

In [4]:
training_length = math.floor(len(df) * 0.7)
train_df, test_df = df.iloc[:training_length], df.iloc[training_length:]

In [5]:
train_data = CurrencyDataset(train_df,  seq_length = 5)
test_data = CurrencyDataset(test_df,  seq_length = 5)

In [6]:
train_dl = DataLoader(train_data, batch_size=8)
test_dl = DataLoader(test_data, batch_size=8)

In [103]:
print('Training Set:')
for sample in train_dl:  
    print(sample['data'].size())
    print(sample['target'].size())
    break

Training Set:
torch.Size([8, 5, 1])
torch.Size([8, 1])


In [144]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        predictions = self.linear(lstm_out[:,-1,:])
        return predictions

In [145]:
import torch
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [146]:
rnn = LSTM(1, 4, 1)

In [129]:
input = next(iter(train_dl))
input = input['data']

In [116]:
input.shape

torch.Size([8, 5, 1])

In [134]:
out = rnn(input)

tensor([[0.2693],
        [0.2692],
        [0.2692],
        [0.2692],
        [0.2692],
        [0.2692],
        [0.2692],
        [0.2692]], grad_fn=<AddmmBackward0>)


In [135]:
out.shape

torch.Size([8, 1])

In [97]:
neural = nn.LSTM(1,4,1)
h0 = torch.randn(1,4)
c0 = torch.randn(1,4)
output, (hn, cn) = neural(input, (h0, c0))

In [98]:
input = train_data[0]['data'] # type: ignore
input
h_0 = torch.randn(1,8)
c_0 = torch.randn(1,8)
output = rnn(input)

In [147]:
loss_fn = torch.nn.MSELoss()

In [148]:
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.001, momentum=0.9)

In [152]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    for step, sample in enumerate(train_dl):
        input = sample['data']
        target = sample['target']
        optimizer.zero_grad()

        # Make predictions for this batch
        output = rnn(input)
        loss = loss_fn(output, target)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if step % 8 == 7:
            last_loss = running_loss / 8 # loss per batch
            print('  batch {} loss: {}'.format(step + 1, last_loss))
            tb_x = epoch_index * len(train_dl) + step + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [154]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    rnn.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    rnn.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, data in enumerate(test_dl):
            vinputs = data['data']
            vtarget = data['target']
            voutputs = rnn(vinputs)
            vloss = loss_fn(voutputs, vtarget)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(rnn.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
  batch 8 loss: 285.6800608634949
  batch 16 loss: 21.294253706932068
  batch 24 loss: 181.87126874923706
  batch 32 loss: 129.16587162017822
  batch 40 loss: 13.993345504626632
  batch 48 loss: 3.346037469804287
  batch 56 loss: 107.76296520233154
  batch 64 loss: 107.37187206745148
  batch 72 loss: 2.3675032146275043
  batch 80 loss: 2.038617569953203
  batch 88 loss: 3.8270446211099625
  batch 96 loss: 5.677450679242611
  batch 104 loss: 31.228965401649475
  batch 112 loss: 54.96153926849365
  batch 120 loss: 44.90354323387146
  batch 128 loss: 11.782090663909912
  batch 136 loss: 19.760085582733154
  batch 144 loss: 7.854134619235992
  batch 152 loss: 17.454182982444763
  batch 160 loss: 7.140122145414352
  batch 168 loss: 6.771441638469696
  batch 176 loss: 41.059249341487885
  batch 184 loss: 30.536306381225586
  batch 192 loss: 34.44527816772461
  batch 200 loss: 11.366944670677185
  batch 208 loss: 3.617546886205673
  batch 216 loss: 0.9828535951673985
  batch 224 loss